# Spell Check Indonesia

## Install Gensim

In [1]:
# !pip install --upgrade gensim

## Download Pre-Trained Fasttext Model Indonesia
Pre-trained Fasttext model yang diunduh adalah hasil pelatihan menggunakan metode CBOW dengan *position-weight*, dalam dimensi 300, dengan panjang karakter n-gram sebesar 5, *window size* 5 dan *negatives* 10. Link kumpulan pre-trained model Fasttext dapat diunduh [disini](https://fasttext.cc/docs/en/crawl-vectors.html).

In [2]:
import wget

def bar_progress(current, total, width=80):
    progress_message = "Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total)
     # Don't use print() as it will print in new line every time.
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()

url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz'

# filename = wget.download(url, bar=bar_progress)

## Extract model dengan menggunakan `gunzip`

In [3]:
%%time

import gzip
import shutil

# with gzip.open('cc.id.300.bin.gz', 'rb') as f_in:
#     with open('cc.id.300.bin', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

Wall time: 0 ns


## Import Libraries

In [4]:
from gensim.models import fasttext
from gensim.models.fasttext import load_facebook_model

## Load Fasttext Model using Gensim

In [5]:
%%time

# model = fasttext.load_facebook_model('cc.id.300.bin')

Wall time: 0 ns


### Pada pre-trained model Fasttext terdapat 2.000.000 (dua juta) *vocabulary*

In [6]:
import pickle

with open('models.pkl', 'rb') as file:
    vocab = pickle.load(file)

len(vocab)

2000000

In [7]:
list(vocab)[:10]

[',', '.', '</s>', 'yang', 'dan', '"', 'di', ')', '(', 'dengan']

In [8]:
# with open('models.pkl', 'wb') as file:
#     pickle.dump(vocab, file)

## Create Index for each word in Vocabulary
Buat *dictionary word-rank* pada tiap kata/karakter yang terdapat pada *vocabulary*. 

In [9]:
%%time

words = list(vocab)

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
    
WORDS = w_rank

Wall time: 1.04 s


In [10]:
import itertools

dict(itertools.islice(WORDS.items(), 10))

{',': 0,
 '.': 1,
 '</s>': 2,
 'yang': 3,
 'dan': 4,
 '"': 5,
 'di': 6,
 ')': 7,
 '(': 8,
 'dengan': 9}

## Peter Norvig Spelling Corrector
Salah satu metode paling sederhana untuk Spelling Corrector adalah dengan metode yang diterapkan oleh [Peter Norvig.](https://norvig.com/spell-correct.html)

### Perbedaan
Terdapat perbedaaan pada code Peter Norvig dengan code CPMP untuk penerapan Spell Check. Perbedaannya jika menggunakan **code Peter Norvig dengan menghitung frekuensi dari tiap kata dari kamus**, sedangkan **code CPMP dengan menggunakan ranking dari Word2Vec model**.

### Cara Kerja
Spell Check ini menggunakan Teorema Bayes untuk menemukan *correction c* dengan memilih *probability* terbesar dari semua *probability candidate correction*. Metode tersebut dapat dijabarkan menjadi 4 bagian:
1. **Selection Mechanism**: `argmax`, dengan memilih *candidate* yang memiliki *probability* terbesar.
2. **Candidate Model:** `c ∈ candidates`, didapatkan *candidate corrections c*, untuk dipertimbangkan.
3. **Languange Model**: `P(c)`, *probability* dari kemunculan *candidate corretion c* pada kamus. Pada penerapannya menggunakan kamus Word2Vec model.
4. **Error Model**: `P(w|c)`, *probability* apabila yang diketik adalah kata *w* sedangkan yang dimaksud adalah *c*. Sebagai contoh, *probability* `P(mkan|makan)` lebih tinggi dibandingkan dengan *probability* `P(mkanxxxyz|makan)` yang lebih rendah.

In [11]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

## Spelling Test

In [12]:
correction("kcing")

'kucing'

In [13]:
correction('J4karta')

'Jakarta'

In [14]:
correction('mnyedihknn')

'menyedihkan'

# Slang + Spell Check

In [15]:
import json

with open("source/slang.txt") as f:
    slangS = json.loads(f.read())
    
with open("source/formal.txt") as f:
    formalS = json.loads(f.read())
    
type(slangS), type(formalS)

FileNotFoundError: [Errno 2] No such file or directory: 'source/formal.txt'

In [ ]:
import re

def join_punctuation(seq, characters='.,;?!'):
    characters = set(characters)
    seq = iter(seq)
    current = next(seq)

    for nxt in seq:
        if nxt in characters:
            current += nxt
        else:
            yield current
            current = nxt

    yield current

def slang(T):
    Texts = re.findall(r"[\w']+|[.,!?;]",T)
    
    _spelling = []
    for text in Texts:
        _spelling.append(correction(text))
    
    for index,text in enumerate(_spelling):
        if text in slangS.keys():
            _spelling[index] = slangS[text]
            
    for index,text in enumerate(_spelling):
        if text in formalS.keys():
            _spelling[index] = formalS[text]
        
    
    _text = ' '.join(_spelling)
    _text = re.sub(r' ([^A-Za-z0-9])', r'\1', _text)
    return _text
    
slang('Unyil prg ke jkt, utk ttp men1mba ilmu!')

'Unyil pergi ke jakarta, untuk ttp menimba ilmu!'

# Sinonim

In [ ]:
import json

with open('source/dict.json') as f:
    mydict = json.load(f)

In [ ]:
def getSinonim(word):
    if word in mydict.keys():
        return mydict[word]['sinonim']
    else:
        return []


def getAntonim(word):
    if word in mydict.keys():
        if 'antonim' in mydict[word].keys():
            return mydict[word]['antonim']
    
    return []

In [ ]:
print(getSinonim('senang'))

['aman', 'bahagia', 'bangga', 'berbungabunga ', 'berkenan', 'bungah', 'camar', 'ceria', 'doyan', 'enak', 'gemar', 'gembira', 'girang', 'lapang dada', 'lega', 'makmur', 'meriah', 'nikmat', 'nyaman', 'puas', 'ria', 'riang', 'sejahtera', 'semarak', 'selesa', 'suka', 'sukacita', 'sukaria', 'tenang', 'tenteram']


In [ ]:
print(getSinonim(getAntonim('senang')[0]))

['duka', 'getir ', 'gundah', 'lara', 'masygul', 'menyesak', 'merana', 'pedih', 'pilu', 'prihatin', 'sedu', 'susah hati', 'terharu', 'trenyuh']


# Sources
- https://github.com/louisowen6/NLP_bahasa_resources#pos-tagging
- https://medium.com/@yasirabd/spell-check-indonesia-menggunakan-pre-trained-fasttext-model-14e90a3f1ac0
- https://norvig.com/spell-correct.html

## Kata Baku

In [ ]:
with open('dict.txt', 'r') as f:
    err = f.read()

In [ ]:
a = err.split("\n")
a[1]

'akta|akte'

In [ ]:
test = []
for i in a:
    x = i.split('|')
    test.append(x)

In [ ]:
test[:10]

[['abjad', 'abjat'],
 ['akta', 'akte'],
 ['aktif', 'aktip'],
 ['aktivitas', 'aktifitas'],
 ['amendemen', 'amandemen'],
 ['andal', 'handal'],
 ['analisis', 'analisa'],
 ['antre', 'antri'],
 ['asas', 'azas'],
 ['apak', 'apek']]

In [ ]:
# Nested list to dict
data= { k[0]: k[1:] for k in test }
# convert dict.value list to string
data_exp = {key:''.join(value) for key,value in data.items()}
# reverse keys and values
new_dict = { v:k for k,v in data_exp.items()}

In [ ]:
# export as txt
with open('formal.txt', 'w') as convert_file:
     convert_file.write(json.dumps(new_dict))

## Imporve Slang

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('source/message.csv')

In [7]:
new_df = df.loc[:, ["transformed", "original-for"]]

In [8]:
new_df

,transformed,original-for
0,sampe,sampai
1,dgn,dengan
2,org,orang
3,karna,karena
4,tu,itu
...,...,...
2120,inii,ini
2121,alhamdulillaah,alhamdulillah
2122,bls,balas
2123,disini,di sini


In [12]:
slang = new_df["transformed"]
formal = new_df["original-for"]

In [13]:
ds = {}

for i,y in zip(slang, formal):
    ds[i] = y

In [16]:
import json
with open('slang1.txt', 'w') as convert_file:
     convert_file.write(json.dumps(ds))

In [21]:
with open("slang.txt") as f:
    slang = json.loads(f.read())
    
with open("slang1.txt") as f:
    slang1 = json.loads(f.read())

In [22]:
combine_slang = slang | slang1

In [26]:
len(combine_slang), len(slang), len(slang1)

(2760, 1021, 2032)

In [27]:
import json
with open('slang_corpus.txt', 'w') as convert_file:
     convert_file.write(json.dumps(combine_slang))